In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,List,Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from pydantic import BaseModel,Field
import operator

load_dotenv()
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest')

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EvaluationSchema(BaseModel):
    
    feedback: str = Field(description='Detailed feedbackfor the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)
    
    

In [3]:
class stateFull(TypedDict):
    
    #user input
    essay : str
    
    clarity_feedback : str
    analysis_feedback : str
    thought_feedback : str
    final_feedback : str
    individual_score : Annotated[list[int], operator.add]
    avg_score : float

In [4]:
Structured_model = model.with_structured_output(EvaluationSchema)

In [5]:

def clarity_feedback(state : stateFull):
    essay = state['essay']
    
    prompt = f'Evaluate the language quality of the {essay} and give it the feedback and score from 0 to 10'
    result = Structured_model.invoke(prompt)
    
    return {'clarity_feedback' : result.feedback, 'individual_score': [result.score]}

def thought_feedback(state : stateFull):
    essay = state['essay']
    
    prompt = f'Evaluate the thought quality of the {essay} and give it the feedback and score from 0 to 10'
    result = Structured_model.invoke(prompt)
    
    return {'thought_feedback' : result.feedback, 'individual_score':[result.score]}

def analysis_feedback(state : stateFull):
    essay = state['essay']
    
    prompt = f'Evaluate the analysis quality of the {essay} and give it the feedback and score from 0 to 10'
    result = Structured_model.invoke(prompt)
    
    return {'analysis_feedback' : result.feedback, 'individual_score':[result.score]}

In [6]:
def final_analysis(state : stateFull):
    try:
        print("DEBUG STATE in final_analysis:", state)
        final_score = sum(state['individual_score']) / 3
        prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["clarity_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["thought_feedback"]}'
        overall_feedback = model.invoke(prompt).content
        
    except Exception as e:
        print("ERROR in final_analysis:", e)
        raise
    # avg calculate

    return {'final_feedback': overall_feedback, 'avg_score': final_score}
    

In [7]:
# define graph
graph = StateGraph(stateFull)

# add nodes
graph.add_node('clarity_feedback',clarity_feedback)
graph.add_node('thought_feedback',thought_feedback)
graph.add_node('analysis_feedback', analysis_feedback)
graph.add_node('final_analysis',final_analysis)

# add edge
graph.add_edge(START,'clarity_feedback')
graph.add_edge(START,'thought_feedback')
graph.add_edge(START,'analysis_feedback')
graph.add_edge('clarity_feedback','final_analysis')
graph.add_edge('thought_feedback','final_analysis')
graph.add_edge('analysis_feedback','final_analysis')
graph.add_edge('final_analysis',END)

# compile
workflow = graph.compile()

In [8]:
essay = """India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

In [ ]:
workflow.invoke({'essay':essay})

DEBUG STATE in final_analysis: {'essay': "India in the Age of AI\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched th

{'essay': "India in the Age of AI\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for 

: 